In [1]:
import os
from cloudmesh.common.Shell import Shell
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_theme(style="whitegrid")
import pandas as pd
from tqdm.notebook import tqdm
from cloudmesh.common.util import readfile
from cloudmesh.common.StopWatch import StopWatch 

In [2]:
user = "gregor"
node = "mac"

processors = 4
experiments = 10
maximum = 1024 * 100000
intervals = 10

In [3]:
label = "{usre}-{node}"

delta = int(maximum / intervals)

totals = [64] + list(range(0,maximum, delta))[1:]
points = [int(t/processors) for t in totals]
print (totals)
print(points)

[64, 10240000, 20480000, 30720000, 40960000, 51200000, 61440000, 71680000, 81920000, 92160000]
[16, 2560000, 5120000, 7680000, 10240000, 12800000, 15360000, 17920000, 20480000, 23040000]


In [4]:
StopWatch.systeminfo()

'+------------------+--------------------------------------------------------------------------------------------------+\n| Attribute        | Value                                                                                            |\n|------------------+--------------------------------------------------------------------------------------------------|\n| cpu_count        | 8                                                                                                |\n| mem.active       | 5.6 GiB                                                                                          |\n| mem.available    | 5.9 GiB                                                                                          |\n| mem.free         | 133.9 MiB                                                                                        |\n| mem.inactive     | 5.5 GiB                                                                                          |\n| mem.percent      | 63.0 %    

In [12]:
df = pd.DataFrame(
{"Size": points}
)
df = df.set_index('Size')

In [13]:
os.makedirs("benchmark", exist_ok=True)

experiment_progress = tqdm(range(0, experiments), desc ="Experiment")

experiment = -1
for experiment in experiment_progress:
    exoeriment = experiment + 1

    label="gregor-mac"
    log = f"benchmark/result-{label}-{experiment}.log"
    os.system(f"rm {log}")


    name = points[experiment]
    progress = tqdm(range(0, len(points)), 
                    desc =f"Benchmark {name}",
                    bar_format="{desc:<30} {total_fmt} {r_bar}")
    
    i = -1
    for state in progress:
        i = i + 1
        n = points[i]
        command = f"mpiexec -n {processors} python count-click.py " + \
                  f"--n {n} --max_number 10 --find 8 --label {label} " + \
                  f"--user {user} --node={node} " + \
                  f"| tee -a {log}"
        os.system (command)

    content = readfile(log).splitlines()
    lines = Shell.cm_grep(content, "csv,Result:")
    # print(lines)
    values = []
    times = []

    for line in lines:
        msg = line.split(",")[7]
        t = line.split(",")[4]

        total, overall, trials, find, label = msg.split(" ")
        values.append(int(overall))
        times.append(float(t))
        # print (t, overall)


    #data = pd.DataFrame(values, times, columns=["Values", "Time"])

    #print (data.describe())
    #sns.lineplot(data=data, palette="tab10", linewidth=2.5)
    # df["Size"] = values
    df[f"Time_{experiment}"] = times
    # print(df)


Experiment:   0%|          | 0/10 [00:00<?, ?it/s]

Benchmark 16                   10 | 0/10 [00:00<?, ?it/s]

Benchmark 2560000              10 | 0/10 [00:00<?, ?it/s]

ValueError: Length of values (8) does not match length of index (10)

In [ ]:

df = df.rename_axis(columns="Time")
df
    

In [ ]:
sns.lineplot(data=df, markers=True)

In [ ]:
plt.show()

In [ ]:
plt.savefig(f'benchmark/benchmark-{label}.png')


In [ ]:
plt.savefig(f'benchmark/benchmark-{label}.pdf')

In [ ]:
plt.show()

In [ ]:
dfs = df.stack().reset_index()
dfs = dfs.set_index('Size')
dfs = dfs.drop(columns=['Time'])
dfs = dfs.rename(columns={0:'Time'})
dfs

In [ ]:
sns.scatterplot(data=dfs, x="Size", y="Time")
sns.relplot(x="Size", y="Time", kind="line", data=dfs);

In [ ]:
df.to_pickle(f"benchmar/{label}-df.pkl")